In [12]:
!uv add "autogen-agentchat" "autogen-ext[openai]" "python-dotenv" "autogen-ext[gemini]" "autogen-ext[ollama]"


Resolved 123 packages in 1ms
Audited 120 packages in 0.02ms
Resolved 123 packages in 1ms
Audited 120 packages in 0.02ms


## initialization

In [1]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)
gemini_api_key = os.getenv("GEMINI_KEY")
open_router_api_key = os.getenv("OPENROUTER_KEY")

In [64]:
import httpx
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollama_client = OllamaChatCompletionClient(model="llama3.1:latest")
open_router_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-r1-0528:free",
    api_key=open_router_api_key,
    model_info={
        "family": "deepseek",
        "vision": True,
        "function_calling": True,
        "json_output": False
    },
    http_client=httpx.AsyncClient(trust_env=False)
)
gemini_client = OpenAIChatCompletionClient(
    model="gemini-2.5-flash",
    api_key=gemini_api_key,
    http_client=httpx.AsyncClient(trust_env=False)
)
# Testing model Client.
print((await ollama_client.create([UserMessage(content="What is the capital of France?", source="user")])).content)
print((await open_router_client.create([UserMessage(content="What is the capital of France?", source="user")])).content)
print((await gemini_client.create([UserMessage(content="What is the capital of France?", source="user")])).content)

The capital of France is Paris.
The capital of France is **Paris**. 🇫🇷
The capital of France is **Paris**.


In [66]:
from autogen_agentchat.agents import AssistantAgent
agent = AssistantAgent(name="RocketScientist", model_client=gemini_client)
result = await agent.run(task="Explain the theory of relativity in simple terms.")
print(result.messages[-1].content)

Okay, let's break down the theory of relativity in simple terms, imagining it as two parts: Special Relativity and General Relativity.

### The Core Idea (What Einstein Realized)

Before Einstein, people thought space and time were like a fixed, invisible grid that everything moved through. Einstein showed that's not quite right. Instead, **space and time are not absolute; they are relative to the observer's motion and gravity.** This means different observers can experience space and time differently.

---

### Part 1: Special Relativity (1905)

This part deals with objects moving at constant speeds (no acceleration). It's based on two revolutionary ideas:

1.  **The Laws of Physics are the Same for Everyone (in uniform motion):** If you're in a car moving at a steady speed, and someone else is standing still, you both experience the same physical laws (e.g., gravity pulls things down the same way). There's no "absolute" state of rest.
2.  **The Speed of Light is Constant:** This is t